# Live Data Outline

In [36]:
import finnhub
import pandas as pd
import numpy as np

In [37]:
import os
from dotenv import load_dotenv

load_dotenv()
FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY")

finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

In [38]:
finnhub_client.company_basic_financials("GOOG", "all").keys()

dict_keys(['metric', 'metricType', 'series', 'symbol'])

In [39]:
type(finnhub_client)

finnhub.client.Client

In [40]:
import logging

In [41]:
def get_us_companies(finnhub_client: finnhub.client.Client):
    if not isinstance(finnhub_client, finnhub.client.Client):
        raise TypeError("finnhub_client must be type finnhub.client.Client")
    
    raw_tickers = finnhub_client.stock_symbols("US")
    companies = dict()
    company_names = list()
    for comp in raw_tickers:
        ticker = comp["symbol"]
        companies[ticker] = comp

        # can ignore for US stocks. Stocks like GOOG and GOOGL are separated already
        ticker2 = comp["symbol2"]
        if ticker2:
            raise ValueError("ticker2 available but not used. Data may be incomplete.")

        company_names.append(comp["description"])

    return companies, company_names

In [42]:
companies, company_names = get_us_companies(finnhub_client)

In [43]:
def get_tickers(companies: dict):
    return list(companies.keys())

In [44]:
def get_metric_names(finnhub_client: finnhub.client.Client, ticker:str, when:str = "quarterly") -> list: 
    """
    Retrieve annual or quarterly time series metrics for a given company.

    when (str): 'quarterly' or 'annual'
    """
    metric_names = None

    try:
        metric_names = finnhub_client.company_basic_financials(ticker, "all")
    except KeyError as e:
        raise ValueError(f"{ticker} is an invalid ticker")
    
    try:
        metric_names = metric_names["series"][when].keys()
    except KeyError as e:
        raise ValueError(f"when is invalid. This should be 'annual' or 'quarterly'")
    
    return metric_names
    

In [45]:
def get_trailing_earnings(finnhub_client: finnhub.client.Client, ticker: str, metric: str, start: int = 0, end: int = 4) -> dict:
    """
    Returns the total (unadjusted) earnings over the specified time period.

    ticker (str): stock ticker
    metric (str): relevant metric (e.g. EPS)
    start (int): 0 for the most recent quarter, 1 for the next most recent quarter, ...
    end (int): ending period (non-inclusive)

    Note: start and end are relative to the most recent earnings call.
    """

    if start > end:
        raise ValueError("start period must be less than or equal to end")


    quarterly_series = finnhub_client.company_basic_financials(ticker, 'all')["series"]["quarterly"]

    if metric not in quarterly_series:
        raise ValueError(f"{metric} not in quarterly series for {ticker}")

    quarterly_series = quarterly_series[metric]

    n_periods = len(quarterly_series)
    print(n_periods)
    if end >= n_periods:
        raise ValueError(f"end must be between 0 and {n_periods-1} (inclusive), but was {end}")

    trailing_eps = sum([report['v'] for report in quarterly_series[start:end]])

    return trailing_eps

get_trailing_earnings(finnhub_client, "MSFT", "eps")

149


14.0554

In [46]:
finnhub_client.company_basic_financials("F", "all")["series"]["quarterly"]

{'assetTurnoverTTM': [{'period': '2025-09-30', 'v': 0.6518},
  {'period': '2025-06-30', 'v': 0.6446},
  {'period': '2025-03-31', 'v': 0.6454},
  {'period': '2024-12-31', 'v': 0.6588},
  {'period': '2024-09-30', 'v': 0.6578},
  {'period': '2024-06-30', 'v': 0.6604},
  {'period': '2024-03-31', 'v': 0.6563},
  {'period': '2023-12-31', 'v': 0.6623},
  {'period': '2023-09-30', 'v': 0.6658},
  {'period': '2023-06-30', 'v': 0.6623},
  {'period': '2023-03-31', 'v': 0.6567},
  {'period': '2022-12-31', 'v': 0.6313},
  {'period': '2022-09-30', 'v': 0.6053},
  {'period': '2022-06-30', 'v': 0.5871},
  {'period': '2022-03-31', 'v': 0.5324},
  {'period': '2021-12-31', 'v': 0.5352},
  {'period': '2021-09-30', 'v': 0.5231},
  {'period': '2021-06-30', 'v': 0.5265},
  {'period': '2021-03-31', 'v': 0.4882},
  {'period': '2020-12-31', 'v': 0.4795},
  {'period': '2020-09-30', 'v': 0.4977},
  {'period': '2020-06-30', 'v': 0.4966},
  {'period': '2020-03-31', 'v': 0.5748},
  {'period': '2019-12-31', 'v': 0.598

In [47]:
import requests
from datetime import datetime

eodhb_API_KEY = os.getenv("eodhb_API_KEY")

In [81]:
def get_historical_price(ticker: str):
    url = f'https://eodhd.com/api/eod/{ticker}.US?api_token={eodhb_API_KEY}&fmt=json'
    data = requests.get(url).json()

    return data

period = "2020-12-31"
period = datetime.strptime(period, f"%Y-%m-%d")
data = get_historical_price("MSFT")
data

[{'date': '2024-12-26',
  'open': 439.08,
  'high': 440.94,
  'low': 436.63,
  'close': 438.11,
  'adjusted_close': 434.9018,
  'volume': 8199927},
 {'date': '2024-12-27',
  'open': 434.6,
  'high': 435.22,
  'low': 426.35,
  'close': 430.53,
  'adjusted_close': 427.3773,
  'volume': 18117711},
 {'date': '2024-12-30',
  'open': 426.06,
  'high': 427.55,
  'low': 421.9,
  'close': 424.83,
  'adjusted_close': 421.7191,
  'volume': 13158700},
 {'date': '2024-12-31',
  'open': 426.1,
  'high': 426.73,
  'low': 420.66,
  'close': 421.5,
  'adjusted_close': 418.4135,
  'volume': 13246510},
 {'date': '2025-01-02',
  'open': 425.53,
  'high': 426.07,
  'low': 414.85,
  'close': 418.58,
  'adjusted_close': 415.5148,
  'volume': 16896471},
 {'date': '2025-01-03',
  'open': 421.08,
  'high': 424.03,
  'low': 419.54,
  'close': 423.35,
  'adjusted_close': 420.2499,
  'volume': 16662939},
 {'date': '2025-01-06',
  'open': 428,
  'high': 434.32,
  'low': 425.4774,
  'close': 427.85,
  'adjusted_clos

In [49]:
period - datetime.strptime("1966-07-05", f"%Y-%m-%d")

datetime.timedelta(days=19903)

In [82]:
get_historical_price("F")

[{'date': '2024-12-26',
  'open': 10,
  'high': 10.12,
  'low': 9.98,
  'close': 10.07,
  'adjusted_close': 9.3753,
  'volume': 35922700},
 {'date': '2024-12-27',
  'open': 10.03,
  'high': 10.2,
  'low': 9.98,
  'close': 10.03,
  'adjusted_close': 9.3381,
  'volume': 52899900},
 {'date': '2024-12-30',
  'open': 9.95,
  'high': 9.98,
  'low': 9.82,
  'close': 9.88,
  'adjusted_close': 9.1984,
  'volume': 47116700},
 {'date': '2024-12-31',
  'open': 9.91,
  'high': 10.01,
  'low': 9.84,
  'close': 9.9,
  'adjusted_close': 9.217,
  'volume': 54104200},
 {'date': '2025-01-02',
  'open': 9.91,
  'high': 9.96,
  'low': 9.64,
  'close': 9.65,
  'adjusted_close': 8.9843,
  'volume': 67156200},
 {'date': '2025-01-03',
  'open': 9.69,
  'high': 9.95,
  'low': 9.53,
  'close': 9.88,
  'adjusted_close': 9.1984,
  'volume': 77245900},
 {'date': '2025-01-06',
  'open': 10.06,
  'high': 10.19,
  'low': 9.9,
  'close': 9.92,
  'adjusted_close': 9.2357,
  'volume': 63840800},
 {'date': '2025-01-07',
 

In [84]:
def get_earnings_price(ticker:str, target_date: datetime.Datetime):
    data = get_historical_price(ticker)
    
    hi, lo = len(data), 0
    mid = (hi + lo) // 2

    curr_date = datetime.strptime(data[mid]["date"], f"%Y-%m-%d")
    prev_date = curr_date
    while curr_date != target_date:
        date_diff = (curr_date - target_date).days

        if date_diff < 0:
            lo = mid
        elif date_diff > 0:
            hi = mid
        elif date_diff == 0:
            return data[mid]
        
        mid = (hi + lo) // 2
        curr_date = datetime.strptime(data[mid]["date"], f"%Y-%m-%d")
        print(curr_date)

        if abs((prev_date - curr_date).days) < 5: # leeway for holidays and weekends
            break

        prev_date = curr_date
        


    close_price = data[mid]["close"]
    print(data[mid])

    return close_price

period = datetime.strptime("2007-12-15", f"%Y-%m-%d")
get_earnings_price("F", period)

2025-03-28 00:00:00
2025-02-12 00:00:00
2025-01-21 00:00:00
2025-01-07 00:00:00
2024-12-31 00:00:00
2024-12-27 00:00:00
{'date': '2024-12-27', 'open': 10.03, 'high': 10.2, 'low': 9.98, 'close': 10.03, 'adjusted_close': 9.3381, 'volume': 52899900}


10.03

In [85]:
def get_market_info(finnhub_client: finnhub.client.Client, ticker:str):
    relevant_series = ["eps", "bookValue", "tangibleBookValue"] 

    financial_info = finnhub_client.company_basic_financials(ticker, "all")

    # Invalid ticker
    if len(financial_info["metric"]) == 0:
        raise ValueError(f"Invalid ticker. {ticker} not found")

    # Retrieve market-close TTM PE
    recent_close = finnhub_client.quote(ticker)["c"]
    recent_eps = get_trailing_earnings(finnhub_client, ticker, "eps")

    metrics = dict()
    metrics["recent_pe"] = recent_close / recent_eps

    series_info = financial_info["series"]["quarterly"]
    series = {metric: series_info[metric] for metric in relevant_series}

    target_date = datetime.strptime("2007-12-15", f"%Y-%m-%d")
    series = get_earnings_price(ticker, target_date)

    return metrics, series


get_market_info(finnhub_client, "MSFT")

149
2025-03-28 00:00:00
2025-02-12 00:00:00
2025-01-21 00:00:00
2025-01-07 00:00:00
2024-12-31 00:00:00
2024-12-27 00:00:00
{'date': '2024-12-27', 'open': 434.6, 'high': 435.22, 'low': 426.35, 'close': 430.53, 'adjusted_close': 427.3773, 'volume': 18117711}


({'recent_pe': 34.69911919973818}, 430.53)

In [ ]:
finnhub_client.company_basic_financials("MSFT", "all")["metricType"]

'all'

In [ ]:
get_market_info(finnhub_client, "F")[1]["eps"]

[{'period': '2025-09-30', 'v': 0.6045},
 {'period': '2025-06-30', 'v': -0.009},
 {'period': '2025-03-31', 'v': 0.1174},
 {'period': '2024-12-31', 'v': 0.4533},
 {'period': '2024-09-30', 'v': 0.222},
 {'period': '2024-06-30', 'v': 0.4552},
 {'period': '2024-03-31', 'v': 0.3311},
 {'period': '2023-12-31', 'v': -0.1317},
 {'period': '2023-09-30', 'v': 0.296},
 {'period': '2023-06-30', 'v': 0.4744},
 {'period': '2023-03-31', 'v': 0.4361},
 {'period': '2022-12-31', 'v': 0.3218},
 {'period': '2022-09-30', 'v': -0.2057},
 {'period': '2022-06-30', 'v': 0.1646},
 {'period': '2022-03-31', 'v': -0.7759},
 {'period': '2021-12-31', 'v': 3.0289},
 {'period': '2021-09-30', 'v': 0.4539},
 {'period': '2021-06-30', 'v': 0.1393},
 {'period': '2021-03-31', 'v': 0.8123},
 {'period': '2020-12-31', 'v': -0.7007},
 {'period': '2020-09-30', 'v': 0.5955},
 {'period': '2020-06-30', 'v': 0.2798},
 {'period': '2020-03-31', 'v': -0.5029},
 {'period': '2019-12-31', 'v': -0.4222},
 {'period': '2019-09-30', 'v': 0.106